# 3.3 Modelos ML Supervisado Categórico MostPopular

3.3.1 Preparación del dataset

3.3.2 Regresión Logística

3.3.3 XGBOOST

Con la aplicación de este modelo se busca testar la existencia de un leakage estructural en la diferenciación entre los vídeos mostpopular y aquellos que no lo son.
Dado que al evaluar distintas combinaciones categóricas los modelos alcanzaron un AUC superior al 95%, entendemos que el algoritmo está capturando algún tipo de metadato o huella estructural asignada por la propia API de YouTube durante la extracción de los vídeos. Esto se explica porque una parte del conjunto de datos fue obtenida a través del endpoint videos.list(chart='mostPopular'), lo cual introduce información implícita sobre el origen del vídeo en la variable objetivo.

Para comprobar esta hipótesis, se seleccionaron cuatro variables exógenas —duration_minutes, has_caption, licensed_content y category_id—, todas ellas definidas en el momento de la publicación del vídeo, no dependientes del rendimiento posterior.
A partir de estas variables se probaron dos algoritmos supervisados, Regresión Logística y XGBoost, con el objetivo de evaluar su capacidad predictiva.
En caso de obtener un rendimiento superior al 90% en métricas como el AUC, se confirma la presencia de leakage estructural, es decir, que el modelo aprende a identificar la procedencia del vídeo dentro del dataset y no su verdadera popularidad.

En consecuencia, la variable mostpopular, tal como está definida en este contexto, no sería adecuada como objetivo de modelización predictiva, ya que su valor está determinado por el propio proceso de recolección de datos y no por una relación causal derivada de las características del vídeo.

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from xgboost import XGBClassifier


In [14]:
def predict_and_get_auc(model, X_train, X_test, y_train, y_test):
    # Predicciones en probas
    y_train_prob = model.predict_proba(X_train)[:, 1]
    y_test_prob  = model.predict_proba(X_test)[:, 1]

    # AUC train
    fpr, tpr, _ = roc_curve(y_train, y_train_prob)
    print("AUC train =", round(auc(fpr, tpr), 2))

    # AUC test
    fpr, tpr, _ = roc_curve(y_test, y_test_prob)
    print("AUC test  =", round(auc(fpr, tpr), 2))

In [15]:

df_1=pd.read_parquet("../DATA/PROCESSED/df_TFMYouTube_embeddings_UMAP5_CLUSTERS_UMAP2.parquet")

cols= ['category_id','duration_minutes',"licensed_content","mostpopular","has_caption",]
df_1=df_1[cols]



FileNotFoundError: [Errno 2] No such file or directory: '../DATA/PROCESSED/df_TFMYouTube_embeddings_UMAP5_CLUSTERS_UMAP2.parquet'

In [4]:
features = ["duration_minutes", "has_caption", "licensed_content", "category_id"]
target = "mostpopular"  # cambia por el nombre de tu variable objetivo

X = df_1[features]
y = df_1[target]

# -----------------------------
# Split de entrenamiento / test
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,          # mantiene proporción de clases
    random_state=42
)


In [8]:


# --- columnas ---
num_cols = ["duration_minutes","has_caption", "licensed_content"]
cat_cols = ["category_id"]

# --- pipes por bloque ---
num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
])  # sin escalar para mantenerlo simple

# Aseguramos string antes del OHE y rellenamos NaN
to_str = FunctionTransformer(lambda X: X.astype(str))
cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("to_str", to_str),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))  # simple y robusto
])

# --- preprocesador ---
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

# --- modelo sencillo (compatible con entrada dispersa/densa) ---
clf = LogisticRegression(
    solver="liblinear",  # soporta sparse y binario
    max_iter=1000,
    random_state=42
)

pipeline_simple = Pipeline([
    ("preprocessing", preprocessor),
    ("model", clf),
])

pipeline_simple.fit(X_train, y_train)
y_pred = pipeline_simple.predict(X_test)
y_prob = pipeline_simple.predict_proba(X_test)[:, 1]
predict_and_get_auc(pipeline_simple, X_train, X_test, y_train, y_test)

AUC train = 0.93
AUC test  = 0.92


In [9]:
clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42
)

pipeline_simple = Pipeline([("preprocessing", preprocessor), ("model", clf)])
pipeline_simple.fit(X_train, y_train)

y_pred = pipeline_simple.predict(X_test)
predict_and_get_auc(pipeline_simple, X_train, X_test, y_train, y_test)

AUC train = 0.98
AUC test  = 0.95
